In [1]:
import sys
sys.path.append('network')
import network
import pandas as pd
import copy


colors = {
        'p__Firmicutes_A': '#1B9E77', 
        'p__Bacteroidota': '#D95F02', 
        'p__Firmicutes': '#7570B3', 
        'p__Proteobacteria': '#E7298A', 
        'p__Actinobacteriota': '#66A61E', 
        'p__Firmicutes_C': '#E6AB02', 
        'p__Verrucomicrobiota': '#A6761D', 
        'p__Firmicutes_B': '#E41A1C', 
        'p__Cyanobacteria': '#377EB8', 
        'p__Fusobacteriota': '#FFFF33',
        'other': '#AAAAAA'}



colors_reverse = {}
phy_table = pd.read_csv('data/species_phylum_pair.csv', header=0, index_col=0)
colors_dict = {}
phy_dict = {}
for phy, color in colors.items():
    colors_dict[color] = 0
    colors_reverse[color] = phy
    phy_dict[phy] = []
total_num = phy_table.shape[0]

for i in range(total_num):
    sp = phy_table.iloc[i, 0]
    phy = phy_table.iloc[i, 1]
    #print(phy, sp)
    if phy not in colors.keys():
        phy = 'other'
    colors_dict[colors[phy]] += 1
    phy_dict[phy].append(sp)

rnode = 5
width, origin, polar_df = network.assign_pos(total_num, rnode, margin=0)


In [2]:

def complete_df(layout, phy_dict, colors_reverse):
    col_order = ['id', 'theta', 'r', 'color', 'phy']
    for idx in layout.index:
        color = layout.loc[idx, 'color'] 
        phy = colors_reverse[color]
        #print(color, phy)
        layout.loc[idx, 'id'] = phy_dict[phy].pop()
        layout.loc[idx, 'phy'] = phy
    return layout[col_order]


In [3]:
# test sector_layout
layout = network.sector_layout(colors_dict, polar_df)
complete_df(layout, copy.deepcopy(phy_dict), colors_reverse).to_csv("sector_sp_layout.tsv", sep='\t', index=None)

with open('config.txt', 'w') as fp:
    fp.write("width = {}\norigin = {},{}\nr = {}".format(width, origin[0], origin[1], rnode))